In [ ]:
%%sh
pip install selenium 
pip install webdriver-manager
pip install cloudscraper

In [ ]:
import html5lib, lxml, requests , re, cloudscraper, os, csv, time, sys, random
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:

from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

url = "https://www.logic-immo.com/vente-immobilier-0,1_0/options/groupprptypesids=1"

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
sleep(20)
p_element = driver.find_element_by_id(id_="add-to-selection")
print(p_element)

scraper = cloudscraper.create_scraper()
source= scraper.get(url).text

#source=requests.get("https://scrapeme.live/shop/").text

In [ ]:
random_url = f"https://www.logic-immo.com/vente-immobilier-0,{random.randint(1,23)}_0/options/groupprptypesids={random.randint(1,23)}/page={random.randint(1,23)}"
requests.get(random_url).text

scraper = cloudscraper.create_scraper()
scraper.get(random_url).text

In [ ]:
extacted_data='extacted_data.csv'
header=np.where(os.path.exists(extacted_data),0,1)
with open(extacted_data, 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # Other available data: energy_letter, client_type, client_id, position, media
    #writer.writerow(['list_name','position','price','estate_type','distribution_type','space','id','id_solo','client_id','photos_nb','estate_postalcode','client_postalcode','country','client_type','parent_site','floor_nb','nb_rooms','nb_bedrooms','energy_letter','product_status','media','product_type']) if header else None
    writer.writerow(['price','estate_type','distribution_type','space','id_solo','photos_nb','floor_nb','nb_rooms','nb_bedrooms','estate_postalcode','url2post','region']) if header else None
    hourlylimit_supervisor=0
    for code_region in range(5,24):
        page=0
        time.sleep(10)
        
        annonces=[-1] 
        while(annonces!=[]):

            # to avoid being blocked and having to solve captcha            
            # if (hourlylimit_supervisor>390):
            #   hourlylimit_supervisor=0
            #   sleep(7200)


            # random clicks to avoid a detection of a crawling pattern
            

            random_url = f"https://www.logic-immo.com/vente-immobilier-0,{random.randint(1,23)}_0/options/groupprptypesids={random.randint(1,23)}/page={random.randint(1,23)}"
            requests.get(random_url).text

            # to be gentle with the site and to emulate a human behavior
            time.sleep(15) 

            page+=1
            url = f"https://www.logic-immo.com/vente-immobilier-0,{code_region}_0/options/groupprptypesids=1/page={page}"
            scraper = cloudscraper.create_scraper()
            source= scraper.get(url).text

            soup=bs(source,'lxml')
            annonces=soup.find_all("a", class_="add-to-selection")

            for annonce in annonces:    

                hourlylimit_supervisor+=1

                target_field=str(annonce).split("{")[3]
                target_data=re.findall(r':([0-9]+)', target_field)
                target_data.pop(0)                  #remove position
                target_data.pop()                   #remove media
                if (len(target_data)<9):            # -1 if floor_nb is not defined
                    target_data.insert(6,-1) 
                elif (len(target_data)>9):         
                    target_data.pop()

                target_data+=re.findall(r'.*estate_postalcode.:\'([\d]+)', target_field) # Preserve the estate_postalcode
                
                id=target_field[target_field.find("id")+5:target_field.find("id_")-3]
                url2post="https://www.logic-immo.com/detail-vente-"+id+".htm"
                target_data.append(url2post)
                target_data.append(code_region)
                writer.writerow(target_data)


In [ ]:
"""
Variables: proposed_price(), apport_complementaire
Input: duree_2possession
From databases: id, price, surface, prix_2ref_moyen_m2,val_loc_indicative_m2, url2post,estate_postalcode

other existing information 
        # estate_type
        # distribution_type: 1 location, 2 vente
        # photos_nb
        # floor_nb
        # nb_rooms
        # nb_bedrooms

Might be useful if found: 
        # historique/ the amount deemed necessary for repairs
        # Economy/ Market outlook on real estate
        # Future estimations of demographics, Government Policies

"""


duree_pret_en_Mois=180
interets=1.03
interets_mensuel=interets/1200
assurance_Pret=0.273
caution_PPD,levee_PPD=2500,2500
penalités_RA=0.03

annuite=interets_mensuel/(1-(1+interets_mensuel)**(-duree_pret_en_Mois))


frais_dagence=10000
pourcentage_commission=0.0799
charge=75
taux_fonciere=33.3

taux_occupation=0.8

class Bien():
    def __init__(self,apport_complementaire,price,surface,estate_postalcode,url2post,prix_2ref_moyen_m2,val_loc_indicative_m2):

        self.price=price
        self.surface=surface
        self.estate_postalcode=estate_postalcode
        self.url2post=url2post

        self.apport_complementaire=apport_complementaire

        self.prix_2ref_moyen_m2=int(prix_2ref_moyen_m2)
        self.val_loc_indicative_m2=val_loc_indicative_m2

    market_value=lambda self:self.prix_2ref_moyen_m2*self.surface 
    worth_it=lambda self:self.market_value()>0.9*self.price  #True                            # when acquiring more accurate gov data self.market_value()>0.9*self.price 

    proposed_price=lambda self:  self.market_value()*0.9 #self.price*0.9          # when acquiring more accurate gov data self.market_value()*0.9

    get_apport_aubanque=lambda self:0.2*self.proposed_price()+self.apport_complementaire
    #+pourcentage_commission*(self.proposed_price()-frais_dagence)

    get_emprunte=lambda self:self.proposed_price()-self.get_apport_aubanque()
    get_price_per_m2=lambda self:self.price/self.surface
    get_prix_propose_par_m2=lambda self:self.proposed_price()/self.surface
    get_mensualites_pret=lambda self:self.get_emprunte()*annuite
    get_mensualites_assu=lambda self:self.get_emprunte()*assurance_Pret/1200

    get_loyer_bas=lambda self:self.surface*self.val_loc_indicative_m2+0.6*charge
    get_loyer_haut=lambda self:self.get_loyer_bas()+0.4*charge+taux_fonciere

    
    
    def residuel(self,duree_2possession):
        return self.residuel(duree_2possession-1)*(interets_mensuel+1)-annuite*self.get_emprunte() if duree_2possession>=1 else self.get_emprunte()

    get_capital_rembourse=lambda self,duree_2possession:self.get_emprunte()-self.residuel(duree_2possession)

    get_plus_value=lambda self:0.2*self.proposed_price()
    """
    Assiette pour l'impôt sur le revenu 0% pour une detention de moins de 6 ans, 
                                        6% pour une detention de la 6e à la 21e année,(pour chaque annee apres la 6e)   
                                        4% pour la 22e année
                                        Exonération au delà de la 22e année

    """
    abattement=lambda self,detention_enannee: self.get_plus_value()*np.where(detention_enannee<6,0,np.where(detention_enannee<=21,6,4))*(detention_enannee-6+1)/100     
    get_impots_plus_value=lambda self,duree_2possession:( self.get_plus_value()-self.abattement(np.floor(duree_2possession/12)))*19/100 if duree_2possession<264 else 0 # Exonération au delà de la 22e année de detention

    get_frais_notaire=lambda self:(self.proposed_price()-frais_dagence)*pourcentage_commission # l'historique est indisponible pour le moment pour un np.where(historique=="Ancien",0.0799,0.02)
   
   

    get_revenu_aumois_i=lambda self,duree_2possession:taux_occupation*duree_2possession*self.get_loyer_bas()+(2-penalités_RA)*(self.get_emprunte()-self.residuel(duree_2possession))+self.get_plus_value()-self.get_impots_plus_value(duree_2possession)+duree_2possession*(self.get_mensualites_pret()-taux_fonciere-charge)-self.get_mensualites_assu()-self.get_frais_notaire()-frais_dagence-levee_PPD*2





In [ ]:
import os,sys,shlex,subprocess,argparse,time,pandas
from numpy.random import randint
import numpy as np
from socket import timeout

# id_solo photos_nb floor_nb estate_postalcode url2post
# values=2,[1],[1],1,0,8,None,120
parameter_s_keys=["distribution_type","region","estate_type","nb_rooms","nb_bedrooms","space","price","duree_2possession"]
parser = argparse.ArgumentParser()

for i in range(len(parameter_s_keys)):
    parser.add_argument('-'+parameter_s_keys[i])
namespace = parser.parse_args()
parameters=namespace.__dict__
locals().update(parameters)

#parameters.pop('x',"") x notrequired
#check_all(parameters)

scrapped_parameters=parameters
scrapped_parameters.pop("duree_2possession")
operation=dict(zip(parameter_s_keys, ["==","in","in","==","==",">=",">=","=="]))
query='region>0'+" ".join(" and "+key+" "+operation[key]+" "+str(value) for key,value in scrapped_parameters.items() if value!=None)
data.query(query)


In [ ]:
#!!!!!!!!! Traitement des donnees dont le prix ref n'existe pas

scrapped_data='scrapped_data.csv'
#data=pd.read_csv("C:/Apache24/htdocs/EstatEagle/dataset/"+scrapped_data)
data=pd.read_csv("/content/"+scrapped_data)
data.columns

parameter_s_keys=["distribution_type","region","estate_type","nb_rooms","nb_bedrooms","space","price","duree_2possession"]
values=2,[1],[1],1,0,8,None,120
parameters = dict(zip(parameter_s_keys, values))


scrapped_parameters=parameters
scrapped_parameters.pop("duree_2possession")
operation=dict(zip(parameter_s_keys, ["==","in","in","==","==",">=",">=","=="]))
query='region>0'+" ".join(" and "+key+" "+operation[key]+" "+str(value) for key,value in scrapped_parameters.items() if value!=None)
filtred=data.query(query)

prix_moyen_2vente_m2='dvf-communes-2019.csv'
#data_ref=pd.read_csv("C:/Apache24/htdocs/EstatEagle/dataset/"+prix_moyen_2vente_m2)
data_ref=pd.read_csv(os.getcwd()+'/'+prix_moyen_2vente_m2)

lines=data_ref['ID;INSEE_COM;INSEE_DEP;INSEE_REG;CODE_EPCI;NOM_COM_M;POPULATION;Nb_Ventes;PrixMoyen_M2']

info={}
for i in lines:
  prix=i[i.rfind(";")+1:]
  info[i[18:23]]=int(prix) if (prix!='NA') else 0


filtred['prix_2ref_moyen_m2']=[info.get(str(i),0) for i in filtred['estate_postalcode']]

#filtred['market_price']=[np.where(int(info.get(str(i),'0'))*j  for i,j in zip(filtred['estate_postalcode'],filtred['space']) ]
#second_query='market_price >+0.9*price'
#second_filtred=filtred.query(second_query)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
### to test 
distribution_type,region,estate_type,nb_rooms,nb_bedrooms,surface,price,duree_2possession=2,1,1,1,0,8,None,120

In [ ]:


"""
Bien inputs: 
        market_value,apport_complementaire,price,surface,estate_postalcode,url2post,prix_2ref_moyen_m2,val_loc_indicative_m2

UI exec passed parameters: 
        distribution_type,region,estate_type,nb_rooms,nb_bedrooms,space,price,duree_2possession

dataframe columns:(scrapped_data.csv + dvf-communes-2019) filtred.iterrows()
        still exist:
        ####################################price                                                             
        estate_type                                                           
        distribution_type                                                    
        ####################################space                                                                 
        id_solo                                                        
        photos_nb                                                            
        floor_nb                                                              
        nb_rooms                                                              
        nb_bedrooms                                                           
        estate_postalcode                                                 
        ####################################url2post              
        region                                                                
        ####################################prix_2ref_moyen_m2
"""
def final():
  for i in filtred.iterrows():
      yield (Bien(40000 ,i[1][0],i[1][3],i[1][9],i[1][10],i[1][12],22)).get_(duree_2possession), (Bien(40000 ,i[1][0],i[1][3],i[1][9],i[1][10],i[1][12],22)).worth_it()

filtred['gain']=[i[0] for i in final()]
filtred['worthit']=[i[1] for i in final()]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1      False
3      False
4      False
5      False
7      False
       ...  
319    False
338    False
342    False
380    False
396    False
Name: worthit, Length: 68, dtype: bool